In [1]:
# import dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [2]:
# get data from database
Base = automap_base()
engine = create_engine('sqlite:///../data/fbdata.sqlite')
Base.metadata.create_all(engine)
Base.prepare(engine, reflect=True)
session = Session(engine)
modelRecent = pd.read_sql_query("SELECT * FROM RecentQBs", con=engine)

In [3]:
# load in model
from tensorflow.keras.models import load_model
model = load_model("fullModel.h5")

from joblib import dump, load
X_scaler=load('X_scaler.bin')

In [5]:
# show model summary
model.summary()

Model: "sequential_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_156 (Dense)            (None, 7)                 49        
_________________________________________________________________
dense_157 (Dense)            (None, 7)                 56        
_________________________________________________________________
dense_158 (Dense)            (None, 7)                 56        
_________________________________________________________________
dense_159 (Dense)            (None, 2)                 16        
Total params: 177
Trainable params: 177
Non-trainable params: 0
_________________________________________________________________


In [6]:
# test with recent data
X_2020 = modelRecent[modelRecent['DraftYear'] >= 2019][['height','40yard','vertleap','rushAvg', 'adjYdsPerAtt','effRate']]
X_2020_scaled = X_scaler.transform(X_2020)
players_2020 = modelRecent[modelRecent['DraftYear'] >= 2019][['Player','DraftYear']].reset_index()
del players_2020['index']

predictions = model.predict_classes(X_2020_scaled)
for i in range(0,len(players_2020)):
    print(f"Prediction for {players_2020['Player'][i]} ({players_2020['DraftYear'][i]}): {predictions[i]}")
    print("-------------------------------------------------------")

Prediction for Kyler Murray (2019): 1
-------------------------------------------------------
Prediction for Daniel Jones (2019): 1
-------------------------------------------------------
Prediction for Dwayne Haskins (2019): 1
-------------------------------------------------------
Prediction for Drew Lock (2019): 1
-------------------------------------------------------
Prediction for Will Grier (2019): 0
-------------------------------------------------------
Prediction for Ryan Finley (2019): 1
-------------------------------------------------------
Prediction for Jarrett Stidham (2019): 0
-------------------------------------------------------
Prediction for Clayton Thorson (2019): 0
-------------------------------------------------------
Prediction for Gardner Minshew (2019): 0
-------------------------------------------------------
Prediction for Trace McSorley (2019): 0
-------------------------------------------------------
Prediction for Joe Burrow (2020): 0
-----------------